In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
%matplotlib inline

URL

In [4]:
urlConfirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
urlDeaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
urlCountries = 'https://restcountries.eu/rest/v2/all'
urlTemp = 'https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_3.23/crucy.1506241137.v3.23/countries/tmp/crucy.v3.23.1901.2014.{}.tmp.per'

Confirmed Cases

In [53]:
df_cc = pd.read_csv(urlConfirmed)

In [54]:
indexesConfirmed = np.arange(2, len(df_cc.columns) - 1)
df_cc.drop(df_cc.columns[indexesConfirmed], axis=1, inplace=True)
df_cc.drop(df_cc.columns[0], axis=1, inplace=True)
df_cc.columns = ['Name','Cases']

In [55]:
df_cc

,Name,Cases
0,Afghanistan,14525
1,Albania,1122
2,Algeria,9267
3,Andorra,764
4,Angola,84
...,...,...
261,Sao Tome and Principe,479
262,Yemen,310
263,Comoros,106
264,Tajikistan,3807


Deaths

In [56]:
df_de = pd.read_csv(urlDeaths)

In [57]:
indexesDeaths = np.arange(2, len(df_de.columns) - 1)
df_de.drop(df_de.columns[indexesDeaths], axis=1, inplace=True)
df_de.drop(df_de.columns[[0, 1]], axis=1, inplace=True)
df_de.columns = ['Deaths']

In [58]:
df_de

,Deaths
0,249
1,33
2,646
3,51
4,4
...,...
261,12
262,77
263,2
264,47


Population

In [73]:
r = requests.get(urlCountries)
arrCountries = r.json()
names = []
populations = []
areas = []
dds = []
lats = []
longs = []
codes = []

for country in arrCountries:
    name = country["name"]
    population = country["population"]
    area = country["area"]
    latlng = country["latlng"]
    code = country["alpha3Code"]
    
    if name == None or population == None or area == None or latlng == None or code == None:
        continue
        
    names.append(name)
    populations.append(population)
    areas.append(area)
    dds.append((population/area))
    lats.append(latlng[0])
    longs.append(latlng[1])
    codes.append(code)
    
dictCountries = {"Name": names, "Code": codes, "Latitude":lats, "Longitude":longs, "Population": population, "Area": areas, "Demographic_Density": dds}
df_co = pd.DataFrame.from_dict(dictCountries)

In [74]:
df_co

,Name,Code,Latitude,Longitude,Population,Area,Demographic_Density
0,Afghanistan,AFG,33.000000,65.0,14240168,652230.0,42.403976
1,Åland Islands,ALA,60.116667,19.9,14240168,1580.0,18.275316
2,Albania,ALB,41.000000,20.0,14240168,28748.0,100.390497
3,Algeria,DZA,28.000000,3.0,14240168,2381741.0,16.962382
4,American Samoa,ASM,-14.333333,-170.0,14240168,199.0,286.934673
...,...,...,...,...,...,...,...
235,Wallis and Futuna,WLF,-13.300000,-176.2,14240168,142.0,82.746479
236,Western Sahara,ESH,24.500000,-13.0,14240168,266000.0,1.919974
237,Yemen,YEM,15.000000,48.0,14240168,527968.0,52.044821
238,Zambia,ZMB,-15.000000,30.0,14240168,752612.0,21.171444


ICU BEDS

In [68]:
df_ib = pd.read_csv('DP_LIVE_31052020055440514.csv')

In [69]:
df_ib.drop(df_ib.columns[[1, 2, 3, 4, 7]], axis=1, inplace=True)
df_new = []

last = df_ib.iloc[0, 0]

for i in range(0, len(df_ib.index)):
    if df_ib.iloc[i, 0] != last:
        new = []
        new.append(df_ib.iloc[i - 1, 0])
        new.append(df_ib.iloc[i - 1, 2])
        
        df_new.append(new)
        last = df_ib.iloc[i, 0]
        
df_ib = pd.DataFrame(df_new, columns = ['Code', 'ICU_Beds'])

In [70]:
df_ib

,Code,ICU_Beds
0,AUS,3.84
1,AUT,7.37
2,BEL,5.64
3,CAN,2.50
4,CZE,6.63
5,DNK,2.50
6,FIN,3.28
7,FRA,5.98
8,DEU,8.00
9,GRC,4.21


# MESCLANDO

In [86]:
df_final = df_cc
df_final['Deaths'] = df_de['Deaths']
df_final = df_final.join(df_co.set_index('Name'), on='Name')
#df_final = df_final.join(df_ib.set_index('Code'), on='Code')
df_final.dropna()

,Name,Cases,Deaths,Code,Latitude,Longitude,Population,Area,Demographic_Density
0,Afghanistan,14525,249,AFG,33.000000,65.00,14240168.0,652230.0,42.403976
1,Albania,1122,33,ALB,41.000000,20.00,14240168.0,28748.0,100.390497
2,Algeria,9267,646,DZA,28.000000,3.00,14240168.0,2381741.0,16.962382
3,Andorra,764,51,AND,42.500000,1.50,14240168.0,468.0,166.696581
4,Angola,84,4,AGO,-12.500000,18.50,14240168.0,1246700.0,20.749178
...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,479,12,STP,1.000000,7.00,14240168.0,964.0,194.352697
262,Yemen,310,77,YEM,15.000000,48.00,14240168.0,527968.0,52.044821
263,Comoros,106,2,COM,-12.166667,44.25,14240168.0,1862.0,432.950054
264,Tajikistan,3807,47,TJK,39.000000,71.00,14240168.0,143100.0,60.053110


In [85]:
df_final

,Name,Cases,Deaths,Code,Latitude,Longitude,Population,Area,Demographic_Density
0,Afghanistan,14525,249,AFG,33.000000,65.00,14240168.0,652230.0,42.403976
1,Albania,1122,33,ALB,41.000000,20.00,14240168.0,28748.0,100.390497
2,Algeria,9267,646,DZA,28.000000,3.00,14240168.0,2381741.0,16.962382
3,Andorra,764,51,AND,42.500000,1.50,14240168.0,468.0,166.696581
4,Angola,84,4,AGO,-12.500000,18.50,14240168.0,1246700.0,20.749178
...,...,...,...,...,...,...,...,...,...
261,Sao Tome and Principe,479,12,STP,1.000000,7.00,14240168.0,964.0,194.352697
262,Yemen,310,77,YEM,15.000000,48.00,14240168.0,527968.0,52.044821
263,Comoros,106,2,COM,-12.166667,44.25,14240168.0,1862.0,432.950054
264,Tajikistan,3807,47,TJK,39.000000,71.00,14240168.0,143100.0,60.053110
